In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
import logging
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

In [ ]:
%%time
preproced_short_X = np.load('/home/schirrmr/code/auto-diagnosis/preproced-short-X.npy')

In [ ]:
%%time
x = preproced_short_X[1]
print(x.shape)
import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.util import np_to_var
window_len = 25
threshold = 10
expected = 1
cuda = True
factor = np.float32(float(expected) / threshold) 

x_var = np_to_var([x])
if cuda:
    x_var = x_var.cuda()

maxs = F.max_pool1d(x_var,window_len, stride=1)
mins = F.max_pool1d(-x_var,window_len, stride=1)

diffs = maxs + mins
print(diffs.size())
large_diffs = (diffs > threshold).type_as(diffs) * diffs
padded = F.pad(large_diffs.unsqueeze(0), (window_len-1,window_len-1, 0,0), 'constant', 0)
max_diffs = th.max(padded[:,:,:,window_len-1:], padded[:,:,:,:-window_len+1]).unsqueeze(0)

x_var = x_var * (expected / max_diffs)
x = x_var.data.cpu().numpy()
print(padded.size())
print(max_diffs.size())
print(x.shape)

In [ ]:
from autoeeglukas.utils.my_io import (time_key, read_all_file_names,
                                      get_recording_length, get_info_with_mne,
                                      load_data_with_mne)
from autoeeglukas.preprocessing.recording import Recording

In [ ]:
normal_file_names = read_all_file_names('/home/gemeinl/data/normal_abnormal/normalv1.1.0/v1.1.0/edf/train/normal/', '.edf',
                    key='time')

In [ ]:
abnormal_file_names = read_all_file_names('/home/gemeinl/data/normal_abnormal/abnormalv1.1.0/v1.1.0/edf/train/', '.edf',
                    key='time')

In [ ]:
import numpy as np

In [ ]:
all_file_names = normal_file_names + abnormal_file_names

all_file_names = sorted(all_file_names,key=time_key)

abnorm_counts = [fname.count('abnormal') for fname in all_file_names]
assert set(abnorm_counts) == set([1,3])
labels = np.array(abnorm_counts) == 3
labels = labels.astype(np.int64)

In [ ]:
%%time
#lengths = [get_recording_length(fname) for fname in all_file_names]

#np.save('sorted-recording-lengths.npy', lengths)# then move somewhere

#lengths = np.array(lengths)

In [ ]:
lengths = np.load('/home/schirrmr/code/auto-diagnosis/sorted-recording-lengths.npy')

In [ ]:
mask = lengths < 35 * 60

In [ ]:
cleaned_file_names = np.array(all_file_names)[mask]
cleaned_labels = labels[mask]

#### just for check what unclean labels are

In [ ]:
unclean_file_names = np.array(all_file_names)[~mask]

unclean_abnorm_counts = [fname.count('abnormal') for fname in unclean_file_names]
unclean_labels = np.array(unclean_abnorm_counts) == 3
print(len(unclean_labels))
print(np.mean(unclean_labels))

In [ ]:
%%time
all_ch_names = [get_info_with_mne(fname)[4] for fname in all_file_names]

In [ ]:
%%time
all_fs = [get_info_with_mne(fname)[1] for fname in all_file_names]
cleaned_fs = np.array(all_fs)[mask]

In [ ]:
cd /home/schirrmr/

In [ ]:
import resampy
from braindecode.datautil.signalproc import exponential_running_standardize
def load_data(fname, preproc_functions):
    cnt, sfreq, n_samples, n_channels, chan_names, n_sec = get_info_with_mne(
        fname)
    log.info("Load data...")
    cnt.load_data()

    wanted_elecs = ['A1', 'A2', 'C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1',
                    'FP2', 'FZ', 'O1', 'O2',
                    'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']

    selected_ch_names = []
    for wanted_part in wanted_elecs:
        wanted_found_name = []
        for ch_name in cnt.ch_names:
            if ' ' + wanted_part + '-' in ch_name:
                wanted_found_name.append(ch_name)
        assert len(wanted_found_name) == 1
        selected_ch_names.append(wanted_found_name[0])

    cnt = cnt.pick_channels(selected_ch_names)

    assert np.array_equal(cnt.ch_names, selected_ch_names)
    data = (cnt.get_data() * 1e6).astype(np.float32)
    fs = cnt.info['sfreq']
    log.info("Preprocessing...")
    for fn in preproc_functions:
        log.info(fn)
        data, fs = fn(data, fs)
        data = data.astype(np.float32)
        fs = float(fs)
    return data
    

In [ ]:
preproc_functions = []
sampling_freq = 100
sec_to_cut = 60
preproc_functions.append(lambda data,fs: (data[:, int(sec_to_cut * fs):-int(
        sec_to_cut * fs)], fs))
preproc_functions.append(lambda data,fs: (np.clip(data,-800,800), fs))
preproc_functions.append(lambda data, fs: (resampy.resample(data, fs,
                                sampling_freq, axis=1, filter='kaiser_fast'), sampling_freq))


In [ ]:
%%time
n_recordings = 100
X = [load_data(fname, preproc_functions) for fname in cleaned_file_names[:n_recordings]]

In [ ]:
np.median([np.std(x) for x in X])

In [ ]:
np.mean([np.std(x) for x in X])

In [ ]:
np.median([np.mean(x) for x in X])

In [ ]:
np.mean([np.mean(x) for x in X])

In [ ]:
plt.plot([np.var(x) for x, this_y in zip(X, cleaned_labels) if this_y== 0])
plt.plot([np.var(x) for x, this_y in zip(X, cleaned_labels) if this_y== 1])


In [ ]:
a = [np.var(x) for x, this_y in zip(X, cleaned_labels) if this_y== 0]
b = [np.var(x) for x, this_y in zip(X, cleaned_labels) if this_y== 1]
print(np.median(a))
print(np.median(b))

In [ ]:
print(np.mean(a))
print(np.mean(b))

In [ ]:
%%time
n_recordings = None
X = [load_data(fname, preproc_functions=[]) for fname in cleaned_file_names[:n_recordings]]

In [ ]:
max_vals = [np.max(x) for x in X]

In [ ]:
min_vals = [np.min(x) for x in X]

In [ ]:
stds =  [np.std(x) for x in X]

In [ ]:
plt.plot(stds)
plt.figure()

seaborn.distplot(stds)

In [ ]:
means =  [np.mean(x) for x in X]

In [ ]:
plt.plot(means)
plt.figure()

seaborn.distplot(means)

In [ ]:
stds_clean =  [np.std(np.clip(x,-800,800)) for x in X]

In [ ]:
plt.plot(stds_clean)
plt.figure()

seaborn.distplot(stds_clean)

In [ ]:
means_clean =  [np.mean(np.clip(x,-800,800)) for x in X]

In [ ]:
plt.plot(means_clean)
plt.figure()

seaborn.distplot(means_clean)

In [ ]:
np.argmax(means_clean)

In [ ]:
standardized = exponential_running_standardize(X[833].T,factor_new=0.001,init_block_size=100).T

In [ ]:
np.max(standardized)

In [ ]:
short_X = [x[:,fs*60:4*fs*60] for x, fs in zip(X, cleaned_fs)]

In [ ]:
apply preproc_functions to short X, also copy thme here maybe change max min to 800...
compute max mean abs etc.

In [ ]:
preproc_functions = []
sampling_freq = 100
preproc_functions.append(lambda data,fs: (np.clip(data,-800,800), fs))
preproc_functions.append(lambda data, fs: (resampy.resample(data, fs,
                                sampling_freq, axis=1, filter='kaiser_fast'), sampling_freq))
preproc_functions.append(lambda data, fs: (exponential_running_standardize(
    data.T,factor_new=0.001).T, fs))


In [ ]:
preproc_functions

In [ ]:
preproced_short_X = []

for x, fs in zip(short_X, cleaned_fs):
    log.info("Preprocess...")
    for fn in preproc_functions:
        x, fs = fn(x, fs)
        x = x.astype(np.float32)
    preproced_short_X.append(x)
        

In [ ]:
preproced_short_X = preproced_short_X[2::3]

In [ ]:
import gc
gc.collect()

In [ ]:
del X

In [ ]:
maxes = [np.max(x) for x in preproced_short_X]
plt.plot(maxes)
plt.figure()
seaborn.distplot(maxes)

In [ ]:
mins = [np.min(x) for x in preproced_short_X]
plt.plot(mins)
plt.figure()
seaborn.distplot(mins)

In [ ]:
np.save('code/auto-diagnosis/preproced-short-X.npy', preproced_short_X)

In [ ]:
np.argsort(maxes)[::-1][10]

In [ ]:
x = preproced_short_X[276]

In [ ]:
np.argmax(np.max(x, axis=0))

In [ ]:
import torch

In [ ]:
x = preproced_short_X[276]

In [ ]:
maxs = torch.nn.functional.max_pool1d(torch.autograd.Variable(
    torch.from_numpy(np.array(preproced_short_X[276:277]))), 10, stride=1).data.cpu().numpy()
mins = -torch.nn.functional.max_pool1d(torch.autograd.Variable(
    torch.from_numpy(-np.array(preproced_short_X[276:277]))), 10, stride=1).data.cpu().numpy()
                                   

In [ ]:
diffs.shape

In [ ]:
diffs = maxs-mins

In [ ]:
np.max(diffs, axis=1).shape

In [ ]:
np.flatnonzero(np.max(diffs, axis=1).squeeze() > 15)

In [ ]:
# make to mask with 10 next ones also in mask

In [ ]:
plt.plot(x[:,17000:18000].T);

In [ ]:
np.maximum.accumulate(x, axis=1)


In [ ]:
np.max(x)

In [ ]:
plt.plot(x[:,17000:18000].T);

In [ ]:
plt.plot(x[:,13840:13880].T);

In [ ]:
x = preproced_short_X[1739]

In [ ]:
np.argmax(np.max(x, axis=0))

In [ ]:
plt.plot(x[:,13000:14000].T);

In [ ]:
plt.plot(x[:,13840:13880].T);

In [ ]:
plt.plot(x[:,7400:7450].T);

In [ ]:
plt.plot([np.percentile(np.abs(x),99) for x in X])

In [ ]:
seaborn.distplot(lengths[lengths < 35 * 60] / 60.0)

In [ ]:
plt.plot(X[0][:,:10000].T);

In [ ]:
y = cleaned_labels[:n_recordings]

In [ ]:
median_amps = []
window_len = 100
for x in X:
    log.info("Compute FFT for recording...")
    this_bps = []
    for i_start in range(0, x.shape[1]-100,100):
        this_bps.append(np.abs(np.fft.rfft(x[:,i_start:i_start+100], axis=1)))
    
    median_amps.append(np.median(this_bps, axis=0))

In [ ]:
median_amps = np.array(median_amps)

In [ ]:
X[0].dtype == 'float32'

In [ ]:
plt.plot(np.mean(median_amps[y==0], axis=(0,1)).T);
plt.plot(np.mean(median_amps[y==1], axis=(0,1)).T);

In [ ]:

from braindecode.datautil.signal_target import SignalAndTarget
from braindecode.datautil.splitters import split_into_two_sets

train_set = SignalAndTarget(X[:60], y=y[:60])
test_set = SignalAndTarget(X[60:], y=y[60:])

train_set, valid_set = split_into_two_sets(train_set, first_set_fraction=0.8)

In [ ]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = 600
n_classes = 2
in_chans = train_set.X[0].shape[0]
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length,
                        final_conv_length=20).create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()

In [ ]:
from torch import optim

optimizer = optim.Adam(model.parameters())


In [ ]:
from braindecode.torch_ext.util import np_to_var
# determine output size
test_input = np_to_var(np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))

In [ ]:
from braindecode.experiments.experiment import Experiment
from braindecode.datautil.iterators import CropsFromTrialsIterator
from braindecode.experiments.monitors import RuntimeMonitor, LossMonitor, CroppedTrialMisclassMonitor, MisclassMonitor
from braindecode.experiments.stopcriteria import MaxEpochs
import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.modules import Expression
# Iterator is used to iterate over datasets both for training
# and evaluation
iterator = CropsFromTrialsIterator(batch_size=32,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

# Loss function takes predictions as they come out of the network and the targets
# and returns a loss
loss_function = lambda preds, targets: F.nll_loss(th.mean(preds, dim=2)[:,:,0], targets)

# Could be used to apply some constraint on the models, then should be object
# with apply method that accepts a module
model_constraint = None
# Monitors log the training progress
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(input_time_length), RuntimeMonitor(),]
# Stop criterion determines when the first stop happens
stop_criterion = MaxEpochs(20)
exp = Experiment(model, train_set, valid_set, test_set, iterator, loss_function, optimizer, model_constraint,
          monitors, stop_criterion, remember_best_column='valid_misclass',
          run_after_early_stop=True, batch_modifier=None, cuda=cuda)

In [ ]:
exp.run()